# LSCP for selecting new sites for all MSOA

Author: Huanfa Chen

In [18]:
import datetime
print("Last update:", datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

Last update: 19/07/2022 01:00:08


In [2]:
# !pip install geopandas
# !pip install pulp

In [1]:
# %time
import numpy
import geopandas 
import pandas
import pulp
from shapely.geometry import Point
import matplotlib.pyplot as plt
# from google.colab import files
import spopt
from spopt.locate.coverage import LSCP
import time

/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Import data

In [2]:
# the service distance in metres (equal to 10 miles)
service_dist = 16093.4
# the distance greater than service distance
great_dist = 20000

In [3]:
# import distance data
# the distance between existing sites and covered MSOAs
df_distance_existing_sites_covered_MSOA = pandas.read_csv('../Data/distance_df_existing_sites_MSOA.csv')

# the distance between potential sites and uncovered MSOAs
df_distance_potential_sites_all_MSOA = pandas.read_csv('../Data/distance_df_potential_sites_all_MSOA.csv')

In [22]:
df_distance_existing_sites_covered_MSOA.head(10)

,Unnamed: 0,origin_id,dest_id,distance
0,122,E02002536,E122,6712.7
1,127,E02002536,E127,13881.2
2,137,E02002536,E137,13631.4
3,836,E02002536,E836,12395.3
4,838,E02002536,E838,12672.0
5,843,E02002536,E843,14449.4
6,844,E02002536,E844,14914.6
7,846,E02002536,E846,14449.4
8,849,E02002536,E849,14914.6
9,1722,E02002537,E122,7828.7


In [38]:
#总共多少个existing sites: 1600
df_distance_existing_sites_covered_MSOA.nunique()

Unnamed: 0    373286
origin_id       6408
dest_id         1600
distance      122263
dtype: int64

In [23]:
df_distance_potential_sites_all_MSOA.head(10)

,origin_id,dest_id,distance
0,E02002536,P14,7041.3
1,E02002536,P207,14950.7
2,E02002536,P354,2963.1
3,E02002537,P14,8157.3
4,E02002537,P207,16066.6
5,E02002537,P354,3153.8
6,E02002534,P123,13144.8
7,E02002534,P162,14661.7
8,E02002534,P207,12390.9
9,E02002535,P14,8752.8


In [41]:
#总共多少个potential sites: 21127
df_distance_potential_sites_all_MSOA.nunique()

origin_id      6788
dest_id       21127
distance     157528
dtype: int64

In [24]:
print(df_distance_existing_sites_covered_MSOA.columns)
print(df_distance_potential_sites_all_MSOA.columns)

Index(['Unnamed: 0', 'origin_id', 'dest_id', 'distance'], dtype='object')
Index(['origin_id', 'dest_id', 'distance'], dtype='object')


In [4]:
# combine two distance df
df_distance_existing_potential_sites_all_MSOAs = pandas.concat([df_distance_existing_sites_covered_MSOA, df_distance_potential_sites_all_MSOA], ignore_index=False)

In [26]:
df_distance_existing_potential_sites_all_MSOAs.head(10)

,Unnamed: 0,origin_id,dest_id,distance
0,122.0,E02002536,E122,6712.7
1,127.0,E02002536,E127,13881.2
2,137.0,E02002536,E137,13631.4
3,836.0,E02002536,E836,12395.3
4,838.0,E02002536,E838,12672.0
5,843.0,E02002536,E843,14449.4
6,844.0,E02002536,E844,14914.6
7,846.0,E02002536,E846,14449.4
8,849.0,E02002536,E849,14914.6
9,1722.0,E02002537,E122,7828.7


## Formulate and solve the LSCP

In [5]:
# transform the distance df to matrix
ntw_dist_piv = df_distance_existing_potential_sites_all_MSOAs.pivot_table(values="distance", index="origin_id", columns="dest_id")
# transform matrix into numpy array
cost_matrix = ntw_dist_piv.to_numpy()

In [7]:
ntw_dist_piv

dest_id,E0,E1,E10,E100,E1000,E1001,E1002,E1003,E1004,E1005,...,P999,P9991,P9992,P9993,P9994,P9995,P9996,P9997,P9998,P9999
origin_id,,,,,,,,,,,,,,,,,,,,,
E02000001,NaN,NaN,NaN,NaN,NaN,5747.1,3668.0,NaN,NaN,NaN,...,10097.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E02006930,NaN,NaN,NaN,NaN,NaN,14784.0,6718.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02006931,NaN,NaN,NaN,NaN,NaN,12446.5,8671.2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02006932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# save the column names as a list
list_site_ID = ntw_dist_piv.columns.to_list()

In [6]:
# if an element is NA or equal to or greater than the service distance in the array, it means it is greater than the predefined service distance. Set it as great_distance
cost_matrix[cost_matrix == service_dist] = great_dist
cost_matrix[numpy.isnan(cost_matrix)] = great_dist

In [9]:
cost_matrix

array([[20000., 20000., 20000., ..., 20000., 20000., 20000.],
       [20000., 20000., 20000., ..., 20000., 20000., 20000.],
       [20000., 20000., 20000., ..., 20000., 20000., 20000.],
       ...,
       [20000., 20000., 20000., ..., 20000., 20000., 20000.],
       [20000., 20000., 20000., ..., 20000., 20000., 20000.],
       [20000., 20000., 20000., ..., 20000., 20000., 20000.]])

In [8]:
cost_matrix.shape

(6788, 22727)

1先实现最基本的lscp的gruobi，基于官方lscp的简单例子（比较快），能独立于pulp
2再多重解，poolsolution，能否运行成功（能输出多个solutions）
3再移植到本个例子里

## 用Gurobi求5000个解

In [8]:
# 设置参数
num_facilities = 22727
num_demand_points = 6788

In [10]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model('facility_location')
# 添加决策变量
select = m.addVars(num_facilities, vtype=GRB.BINARY, name='Select')
# 设置限制条件
    # 每个i在距离x(16093.4)内至少被1个j覆盖
m.addConstrs((gp.quicksum(select[j] for j in range(num_facilities) if cost_matrix[i,j] < 16093.4) >= 1  for i in range(num_demand_points)), name='Demand')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-14


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [11]:
# Limit how many solutions to collect 想要收集多少个解
m.setParam(GRB.Param.PoolSolutions, 5000)
# Limit the search space by setting a gap for the worst possible solution that will be accepted
m.setParam(GRB.Param.PoolGap, 0.10)
# do a systematic search for the k-best solutions
m.setParam(GRB.Param.PoolSearchMode, 2)

Set parameter PoolSolutions to value 5000
Set parameter PoolGap to value 0.1
Set parameter PoolSearchMode to value 2


In [12]:
# 设置模型目标
m.setObjective(gp.quicksum(select[j] for j in range(num_facilities)), GRB.MINIMIZE)

In [13]:
#运行模型
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6788 rows, 22727 columns and 2611068 nonzeros
Model fingerprint: 0x11dd9b8f
Variable types: 0 continuous, 22727 integer (22727 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 426.0000000
Presolve removed 2191 rows and 1 columns
Presolve time: 4.33s
Presolved: 4597 rows, 22726 columns, 2335089 nonzeros
Variable types: 0 continuous, 22726 integer (22724 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+00   4.589000e+03   0.000000e+00      5s
    8942    3.0632568e+02   0.000000e+00   0.000000e+00     10s

Root relaxation: objective 3.063257e+02, 8942 iterations, 4.93 seconds (5.51 work units)

    Nodes    |    C

In [18]:
# Status checking
import sys
status = m.Status

if status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED):
    print('The model cannot be solved because it is infeasible or unbounded')
    sys.exit(1)

if status != GRB.OPTIMAL:
    print('Optimization was stopped with status ' + str(status))
    sys.exit(1)

In [19]:
# Print best selected set
print('Selected elements in best solution:')
print('\t', end='')
for j in range(num_facilities):
    if select[j].X > .9:
        print(' facility%d' % j, end='')
print('')

Selected elements in best solution:
	 facility76 facility104 facility108 facility281 facility451 facility599 facility860 facility876 facility884 facility1104 facility1133 facility1265 facility1287 facility1307 facility1352 facility1376 facility1484 facility1488 facility1518 facility1610 facility1679 facility1702 facility1756 facility1761 facility1763 facility1858 facility1879 facility1959 facility2004 facility2063 facility2232 facility2389 facility2484 facility2509 facility2511 facility2542 facility2587 facility2616 facility2664 facility2751 facility2792 facility2836 facility2847 facility2859 facility3000 facility3005 facility3122 facility3339 facility3515 facility3643 facility3652 facility3656 facility3676 facility3904 facility3928 facility3977 facility3982 facility4091 facility4149 facility4486 facility4507 facility4620 facility4719 facility4772 facility4855 facility4916 facility4936 facility4965 facility4970 facility5020 facility5038 facility5098 facility5104 facility5139 facility52

In [14]:
# Print number of solutions stored
nSolutions = m.SolCount
print('Number of solutions found: ' + str(nSolutions))

Number of solutions found: 5000


In [15]:
# Print objective values of solutions
for e in range(nSolutions):
    m.setParam(GRB.Param.SolutionNumber, e)
    print('%g ' % m.PoolObjVal, end='')
    if e % 15 == 14: #这一行没看懂，但好像不影响输出结果的正确性
        print('')
print('')

313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 

In [22]:
# print fourth best set if available
if (nSolutions >= 4):
    m.setParam(GRB.Param.SolutionNumber, 3)

    print('Selected elements in fourth best solution:')
    print('\t', end='')
    for j in range(num_facilities):
        if select[j].Xn > .9:
            print(' facility%d' % j, end='')
    print('')

Selected elements in fourth best solution:
	 facility76 facility108 facility281 facility346 facility599 facility778 facility798 facility860 facility876 facility882 facility884 facility1130 facility1133 facility1265 facility1287 facility1484 facility1488 facility1518 facility1617 facility1679 facility1702 facility1752 facility1756 facility1761 facility1768 facility1786 facility1858 facility1877 facility1995 facility2063 facility2246 facility2285 facility2389 facility2479 facility2484 facility2509 facility2511 facility2542 facility2587 facility2603 facility2616 facility2664 facility2693 facility2713 facility2751 facility2792 facility2836 facility2847 facility2859 facility3005 facility3122 facility3225 facility3339 facility3512 facility3604 facility3643 facility3652 facility3656 facility3676 facility3693 facility3904 facility3914 facility3928 facility3977 facility3982 facility4091 facility4149 facility4160 facility4417 facility4486 facility4507 facility4719 facility4772 facility4855 facil

In [40]:
if (nSolutions >= 4):
    m.setParam(GRB.Param.SolutionNumber, 3)
    
    #一共1600个existing facilities；计算序号小于等于1599的有多少个
    sum = 0

    for j in range(1600):
        if select[j].Xn >= 1.0:
            sum = sum + 1
    
    print(sum)

18


In [20]:
summary = pandas.DataFrame(columns=['existing_count', 'potential_count'], index=range(5000))

In [21]:
for n in range(5000):
    m.setParam(GRB.Param.SolutionNumber, n)
    
    #一共1600个existing facilities；计算序号小于等于1599的有多少个
    sum = 0
    for j in range(1600):
        if select[j].Xn >= 1.0:
            sum = sum + 1
    summary.iloc[n,0] = sum

In [1]:
summary.max()

NameError: name 'summary' is not defined

In [ ]:
# 结果的list和existing的list作对比，看多少是overlap。可以是表格或者直方图，百分比或者绝对值表示
# 一个总的list，包括所有的解

#有权重的解法，直接用gurobi实现，和手动的去对比
#给权重的数值做一个list，和select相乘

#下周：
#1）手动：所有的解找出来，对比，找出已有最多的解
#2）自动：有权重的模型，和手动结果对比验证一下看是否正确
#3）分析：一些图表

## Extended LSCP模型

In [22]:
import gurobipy as gp
from gurobipy import GRB

m_extended = gp.Model('facility_location_extended')
# 添加决策变量
select_extended = m_extended.addVars(num_facilities, vtype=GRB.BINARY, name='Select')
# 设置限制条件
    # 每个i在距离x(16093.4)内至少被1个j覆盖
m_extended.addConstrs((gp.quicksum(select_extended[j] for j in range(num_facilities) if cost_matrix[i,j] < 16093.4) >= 1  for i in range(num_demand_points)), name='Demand')

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [23]:
# Limit how many solutions to collect
m_extended.setParam(GRB.Param.PoolSolutions, 1)
# Limit the search space by setting a gap for the worst possible solution that will be accepted
m_extended.setParam(GRB.Param.PoolGap, 0)
# do a systematic search for the k-best solutions
m_extended.setParam(GRB.Param.PoolSearchMode, 2)

Set parameter PoolSolutions to value 1
Set parameter PoolGap to value 0
Set parameter PoolSearchMode to value 2


In [24]:
weight = {}
for n in range(1600):
    weight[n] = 99
for n in range(1600, num_facilities):
    weight[n] = 100

In [25]:
# 设置模型目标
# m_extended.setObjective((gp.quicksum(select_extended[j] for j in range(1600)) * 99 + gp.quicksum(select_extended[j] for j in range(1600, num_facilities)) * 100), GRB.MINIMIZE)
# m_extended.setObjective(gp.quicksum(99 * select_extended[j] for j in range(1600)) + gp.quicksum(100 * select_extended[j] for j in range(1600, num_facilities)), GRB.MINIMIZE)

m_extended.setObjective(gp.quicksum(select_extended[j] * weight[j] for j in range(num_facilities)), GRB.MINIMIZE)

In [26]:
#运行模型
m_extended.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6788 rows, 22727 columns and 2611068 nonzeros
Model fingerprint: 0x696a8947
Variable types: 0 continuous, 22727 integer (22727 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 42600.000000
Presolve removed 2191 rows and 1 columns
Presolve time: 4.20s
Presolved: 4597 rows, 22726 columns, 2335089 nonzeros
Variable types: 0 continuous, 22726 integer (22724 binary)
Found heuristic solution: objective 41234.000000

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+02   4.589000e+03   0.000000e+00      5s
    8913    3.0560946e+04   0.000000e+00   0.000000e+00     10s

Root relaxation: objective 3.056095e+04, 8913 iterations, 4

In [13]:
# Print number of solutions stored
nSolutions_extended = m_extended.SolCount
print('Number of solutions found: ' + str(nSolutions_extended))

Number of solutions found: 10


In [14]:
# Print objective values of solutions
for e in range(nSolutions_extended):
    m_extended.setParam(GRB.Param.SolutionNumber, e)
    print('%g ' % m_extended.PoolObjVal, end='')
    if e % 15 == 14:
        print('')
print('')

31193 31193 31196 31196 31197 31198 31200 31296 31297 31602 


In [18]:
summary_extended = pandas.DataFrame(columns=['existing_count', 'potential_count'], index=range(10))

In [19]:
for n in range(10):
    m_extended.setParam(GRB.Param.SolutionNumber, n)
    
    #一共1600个existing facilities；计算序号小于等于1599的有多少个
    sum = 0
    for j in range(1600):
        if select_extended[j].Xn >= 1.0:
            sum = sum + 1
    summary_extended.iloc[n,0] = sum

    #计算potential sites数量
    sum_potential = 0
    for j in range(1600, num_facilities):
        if select_extended[j].Xn >= 1.0:
            sum_potential = sum_potential + 1
    summary_extended.iloc[n,1] = sum_potential

In [20]:
summary_extended

,existing_count,potential_count
0,107,206
1,106,206
2,104,209
3,104,209
4,103,210
5,102,211
6,100,213
7,104,210
8,103,210
9,98,219


In [21]:
summary_extended['total'] = summary_extended['existing_count'] + summary_extended['potential_count']
summary_extended

,existing_count,potential_count,total
0,107,206,313
1,106,206,312
2,104,209,313
3,104,209,313
4,103,210,313
5,102,211,313
6,100,213,313
7,104,210,314
8,103,210,313
9,98,219,317


## Pulp解法

In [14]:
time_start = time.time()
lscp_all_MSOA = LSCP.from_cost_matrix(cost_matrix, service_dist)
lscp_all_MSOA = lscp_all_MSOA.solve(pulp.GUROBI(msg=False, PoolSolutions=10)) #这种写法不对
lscp_all_MSOA.facility_client_array()
print("LSCP solution time (seconds): {}".format(int(time.time() - time_start)))

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-14
LSCP solution time (seconds): 4803


In [15]:
lscp_all_MSOA

In [15]:
# check that the result is correct

# are all demand points covered by at least one selected facility?
lscp_all_MSOA.uncovered_clients()
print("Number of uncovered client: {}".format(lscp_all_MSOA.n_cli_uncov))

# how many facilities are selected
list_fac_sites_selected = [i for i, x in enumerate(lscp_all_MSOA.fac2cli) if x]
print("Number of selected facilities: {}".format(len(list_fac_sites_selected)))
# getting the ID list of selected facilities
list_id_fac_sites_selected = [list_site_ID[i] for i in list_fac_sites_selected]

# save the dataframe of selected facility to a file
# facility_points.loc[facility_points.site_ID.isin(list_id_fac_sites_selected)].to_csv("../Data/df_LSCP_facility_for_uncovered_MSOA.csv", index=False)

Number of uncovered client: 0
Number of selected facilities: 313


In [16]:
# save the dataframe of selected facility to a file
pandas.DataFrame({'site_ID':list_id_fac_sites_selected}).to_csv("../Data/df_LSCP_facility_ID_for_all_MSOAs.csv", index=False) 